<a href="https://colab.research.google.com/github/Que1Pereza2/Mr.CrabsAnalyzer/blob/main/CanYouFeelItNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports block

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

This block reads the None2775.csv file and loads it's contents into review.

In [2]:
reviews = pd.read_csv("None2775.csv")

This block handles the undersampling of the positive reviews so the scores appear in equal quantity and creates the label and features arrays used the neural network to train and test.

In [3]:
# Takes all the reviews[score] values and replaces the ' " ' with no space so
#they can be converted to int.
reviews['score'] = reviews['score'].str.replace('"', '').astype(int)

# Separates all the reviews by score.
majority_class = reviews[reviews.score == 1]
minority_class = reviews[reviews.score == 0]

# Downsample majority class
majority_downsampled = majority_class.sample(n = len(minority_class), random_state=42)

# Combine minority class with downsampled majority class
balanced_df = pd.concat([majority_downsampled, minority_class])

# Shuffle the resulting DataFrame
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Creates the features and labels arrays
features = balanced_df.iloc[:, 0].values
labels = balanced_df.iloc[:, 1].values


This block converts the scores from strings to ints, uses regex to clean the data and vectorizes the labels DataFrame, which is composed of strings, so the Neural Network can train on it.

In [8]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters.
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # Remove all single characters.
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start.
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)

    # Substituting multiple spaces with single space.
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'.
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase.
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

# Creation of the vectorizer.
vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8)

# Loading the vectorized array back into processed_features.
processed_features = vectorizer.fit_transform(processed_features).toarray()

This block splits the data into train and test arrays and feeds the training data to the Neural Network.

In [9]:
# Creation of the test and train arrays.
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

# Creation of the Neural Network.
text_classifier = RandomForestClassifier(criterion="entropy", n_estimators=2000, random_state=42)

# Training the Neural Network.
text_classifier.fit(X_train, y_train)

# Generating the Test results.
predictions = text_classifier.predict(X_test)

Neural Network scores.

In [ ]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))

User Interface.

In [21]:
reviewToPredict = input(f"Please provide a review!\n ")
resultUser = text_classifier.predict(vectorizer.transform([reviewToPredict]).toarray())

if resultUser == 1:
    print("The review is positive")
else:
    print("The review is negative")

Please provide a review!
 trash
The review is negative
